### Build and push image

In [ ]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-tf-profiler

cd container

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-east-1 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email --registry-ids 763104351884 )
#$(aws ecr get-login --region us-east-1 --no-include-email --registry-ids 763104351884)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

In [ ]:
import boto3

account_id = boto3.client("sts").get_caller_identity()["Account"]
region = boto3.Session().region_name
image_uri = "{}.dkr.ecr.{}.amazonaws.com/sagemaker-tf-profiler:latest".format(account_id,region)

In [ ]:
import sagemaker

role = sagemaker.get_execution_role()

### Create Feature Group and ingest 
Below we generate some random input data for our model and ingest it into Feature Store.
We do this as our inference.py script will confirm that the input we are sending to our model for prediction is the latest features we saved in Feature Store

In [ ]:
import pandas as pd
import numpy as np
from time import gmtime, strftime
import time


random_input = np.random.rand(1, 1, 3, 3)
random_input = random_input.tolist()
df = pd.DataFrame()

df["RandomInput"] = [random_input]
df = df.astype({'RandomInput': 'string'})

current_time_sec = int(round(time.time()))
df["EventTime"] = current_time_sec
df = df.astype({'EventTime': 'float64'})
df['Index'] = range(1, len(df) + 1)
df["RandomInput"]


In [ ]:
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.session import Session
sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()

prefix = 'DEMO-CodeGuru-Profiler'
my_features_feature_group = FeatureGroup(
    name="my-features", sagemaker_session=sagemaker_session
)


my_features_feature_group.load_feature_definitions(data_frame=df)

record_identifier_feature_name="RandomInput"

my_features_feature_group.create(
    s3_uri=f"s3://{bucket}/{prefix}",
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name="EventTime",
    role_arn=role,
    enable_online_store=True,
)

In [ ]:
def check_feature_group_status(feature_group):
    status = feature_group.describe().get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group to be Created")
        time.sleep(5)
        status = feature_group.describe().get("FeatureGroupStatus")
    print(f"FeatureGroup {feature_group.name} successfully created.")


check_feature_group_status(my_features_feature_group)

In [ ]:
my_features_feature_group.ingest(data_frame=df, max_workers=3, wait=True)


### Create an inference.py
Here we create an inference.py with an expensive function that retrieves the latest feature value from Feature Store

In [1]:
%%writefile inference.py

import json
import time
import numpy as np
import logging


def retrieve_latest_features_expensive(data,record_id):
    import boto3

    client = boto3.client('sagemaker-featurestore-runtime')

    response = client.get_record(
    FeatureGroupName='my-features',
    RecordIdentifierValueAsString=str(record_id),

    )
    return response["Record"][0]["ValueAsString"]
    

def input_handler(data, context):
    """ Pre-process request input before it is sent to TensorFlow Serving REST API
    Args:
        data (obj): the request data, in format of dict or string
        context (Context): an object containing request and configuration details
    Returns:
        (dict): a JSON-serializable dict that contains request body and headers
    """

    if context.request_content_type == 'application/json':

        d = data.read().decode('utf-8')
      
        input_data = json.loads(d)
        print("Prediction features recieved from InvokeEndpoint API:")
        print(type(input_data["data"]))
        print("Features recieved from Feature Store:")
        print(type(retrieve_latest_features_expensive(input_data["data"],json.loads(input_data["id"]))))
              
        assert input_data["data"] == json.loads(retrieve_latest_features_expensive(input_data["data"],input_data["id"]))
      
        return json.dumps({"inputs" : input_data["data"]})


    raise ValueError('{{"error": "unsupported content type {}"}}'.format(
        context.request_content_type or "unknown"))


def output_handler(data, context):
    """Post-process TensorFlow Serving output before it is returned to the client.
    Args:
        data (obj): the TensorFlow serving response
        context (Context): an object containing request and configuration details
    Returns:
        (bytes, string): data to return to client, response content type
    """
    if data.status_code != 200:
        raise ValueError(data.content.decode('utf-8'))
    response_content_type = context.accept_header
    prediction = data.content
    return prediction, response_content_type

Overwriting inference.py


### Deploy our Model to an Endpoint
Our container has been pushed to ECR and our Model is in S3 now we have everything we need to Deploy to a SageMaker Endpoint.

In [ ]:
%%time
saved_model = (
    "s3://sagemaker-sample-data-{}/tensorflow/model/resnet/resnet_50_v2_fp32_NCHW.tar.gz".format(
        region
    )
)

In [ ]:
from sagemaker.tensorflow.serving import Model
from sagemaker.local import LocalSession

tensorflow_model = Model(model_data=saved_model,
                         role=role,
                         entry_point = "inference.py",
                         image_uri=image_uri,
                         env={"AWS_DEFAULT_REGION": "us-east-1"},
                        sagemaker_session = LocalSession()
                        )

In [ ]:
%%time
predictor = tensorflow_model.deploy(
    initial_instance_count=1, instance_type="ml.t2.medium"
)

In [ ]:
%%time
import numpy as np
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer

input_data = {
    "data": random_input,
    "id" : "1"
}
#for i in range (0,20000):
while True:
    #predictor.serializer = ser
    prediction = predictor.predict(input_data)
    print(prediction)
    break

    